In [ ]:
# Reset and Restart 

import sys
import os
import numpy as np
import time
from imp import reload
import threading
from L4 import FileIO
from skimage import io

# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")
import testing.inputs  
reload(testing.inputs)
from testing.inputs import *
from L4 import DisplayWindows
reload(DisplayWindows)

CONFIG_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Eclipse.cfg"
from L3 import SystemsBuilder
reload(SystemsBuilder)

ce_system = SystemsBuilder.CESystem()

ce_system.load_config(CONFIG_PATH)
ce_system.open_controllers()
ce_system.startup_utilities()

%matplotlib qt

In [ ]:

class ImageSaver:
    
    def __init__(self, system:SystemsBuilder.CESystem, folder_prefix=None, data_folder=None):
        self.system = system
        self.prefix = folder_prefix
        self.data_folder = data_folder
        self.images = []
        self._lock = threading.RLock()
        
    def add_callback(self):
        self.system.camera.add_callback(self.callback, tag='save_img')
        
    def remove_callback(self):
        self.system.camera.remove_callback(tag='save_img')
    
    def callback(self, img, *args, **kwargs):
        with self._lock:
            self.images.append(img)
    
    def save_image(self, image_prefix='img', folder_prefix=None, data_folder=None, unique_folder=True):
        """
        Saves the images into an folder with incrementing image names:
        
        image_prefix: what the image filename should be caleld
        folder_prefix: what the data folder will be called
        data_folder: where the parent_directory resides
        unique_folder: When true, create a new folder everytime the function is called
        
        """
        
        if folder_prefix is None:
            prefix = self.prefix
        if data_folder is None:
            data_folder = self.data_folder
    
        if unique_folder:
            data_dir = FileIO.get_data_folder(prefix, data_folder)
        else:
            data_dir = data_folder
        for image in self.images:
            data_file = FileIO.get_data_filename(image_prefix, data_dir, extension='.tiff')
            io.imsave(data_file, image)
            
    
    

C:\Users\Luke\Anaconda3\envs\CEpy37\lib\site-packages\nidaqmx\task.py:105: ResourceWarning: Task of name "_unnamedTask<0>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  'reserved.'.format(self.name), DaqResourceWarning)
C:\Users\Luke\Anaconda3\envs\CEpy37\lib\site-packages\nidaqmx\task.py:105: ResourceWarning: Task of name "_unnamedTask<1>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  'reserved.'.format(self.name), DaqResourceWarning)
C:\Users\Luke\Anaconda3\envs\CEpy37\lib\site-packages\nidaqmx\task.py:105: ResourceWarning: Task of name "_unnamedTask<2>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  'reserved.'.format(self.name), DaqResourceWarning)


In [3]:
from pathlib import Path
import numpy as np
from scipy.ndimage import label, distance_transform_edt
from skimage import io, img_as_float, filters, morphology
from skimage.feature import peak_local_max
from skimage.measure import regionprops, regionprops_table
from skimage.morphology import watershed
import pandas as pd
import glob
import skimage.measure as measure

import matplotlib.pyplot as plt

In [19]:
from L4 import Focus
reload(Focus)

<module 'L4.Focus' from 'C:\\Users\\Luke\\Desktop\\Barracuda\\AutomatedCE\\L4\\Focus.py'>

# Automation Objects

In [281]:
# Set up the Automation Method & Template 
TEMPLATE_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\EclipseCellTemplate.txt"
METHOD_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Brae\Cell.txt"

from L4 import AutomatedControl
from L4 import Trajectory
reload(AutomatedControl)
reload(Trajectory)

auto_program = AutomatedControl.AutoRun(ce_system)
auto_program.set_template(TEMPLATE_PATH)
auto_program.add_method(METHOD_PATH)
auto_program.repetitions=6

WHASSUZ ['cell\tcells\t240\t7 mm\t0\t0\t0\t3\twait\t0', 'auto_cell\tw_e1\t260\t7 mm\t0\t0\t20\t100\t\t1']
ADDING
ADDING


In [74]:
# Start the Method
if not auto_program.continue_event.is_set():
    auto_program.continue_event.set()
else:
    auto_program.start_run()

After special True
Time to wait is  3
After special True
Time to wait is  200


In [ ]:
# Stop the Method
auto_program.stop_run()

In [328]:
auto_program.start_run()

In [89]:
name = get_value("Please enter the well name to move to (press enter to see a list): ", auto_program.template.wells.keys())
well = auto_program.template.wells[name]
ce_system.xy_stage.set_xy(well.xy)
resp = input("Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.")


Please enter the well name to move to (press enter to see a list): cells
Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.


In [75]:
# CE Display
ce_gram = DisplayWindows.PLTCEDisplay(ce_system)
#ce_disp.stop()
#ce_disp.show()
ce_gram.start_live_view()
ce_system.detector.start()

Exception in thread Thread-31:
Traceback (most recent call last):
  File "C:\Users\Luke\Anaconda3\envs\CEpy37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\Luke\Anaconda3\envs\CEpy37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L2\CameraControl.py", line 196, in _sequence_update
    img = self.controller.send_command(self.controller.core.get_last_image)
  File "C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\Controllers.py", line 227, in send_command
    raise e
  File "C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\Controllers.py", line 219, in send_command
    ans = command(*args)
  File "C:\Users\Luke\Anaconda3\envs\CEpy37\lib\site-packages\pycromanager\core.py", line 263, in <lambda>
    fn = lambda instance, *args, signatures_list=tuple(methods_with_name): instance._translate_call(signatures_list, args)
  File "C:\Users\Luke\Anaconda3\envs\CEpy37\lib\site-pack

In [3]:
# Microscope Display
cam = DisplayWindows.PLTMicroscopeDisplay(ce_system)
ce_system.camera.stop()
cam.show()
cam.live_image()

HEYYOO


# Set the Focus Plane for the cells

In [20]:
focus_plane = Focus.PlaneFocus(ce_system)

focus_plane.get_three_points()


In [23]:
# run this three times
focus_plane.give_response()

In [24]:
focus_plane.find_a_plane()

True

In [25]:
focus_search = Focus.FindFocus(ce_system)

# Set the Capillary Adjustments

In [7]:
# Move the objective to the height and run this cell to record it for next time
obj_height = ce_system.objective.read_z()
ce_system.objective.set_rel_z(+.05)

In [13]:
ce_system.inlet_z.set_rel_z(-0.5)

In [6]:
ce_system.inlet_z.set_z(4)

In [14]:
# and record this height for next time 
cap_height = ce_system.inlet_z.read_z()
cap_difference = obj_height - cap_height

In [15]:
def get_cap_height(obj_height, cap_difference):
    cap_height = obj_height - cap_difference
    return cap_height

In [179]:
import csv

In [180]:
with open('save_me','w') as fout:
    fout.write(f'cap_difference,{cap_difference}')

In [26]:
with open('save_me','r') as fin:
    cap_difference= float(fin.readline().split(',')[1])
    

In [16]:
cap_difference

7.8378251316938545

In [26]:
def get_brightest_blobs(image):
    close_size=5
    open_size=5
    watershed_footprint = (5,5)
    # Make sure types are the same
    input_image = img_as_float(img)

    # Filter Image
    filtered_image = filters.median(input_image, behavior='ndimage')

    # Edge Detection
    edge_sobel = filters.sobel(filtered_image)

    # Threshold
    thresh = filters.threshold_otsu(filtered_image)
    binary_otsu = filtered_image > thresh

    # Binary Morphology Operations
    structure_element = morphology.disk(close_size)
    closed_image = morphology.binary_closing(binary_otsu, structure_element)
    structure_element = morphology.disk(open_size)
    opened_image = morphology.binary_opening(closed_image, structure_element)

    # Watershed
    distance = distance_transform_edt(opened_image)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones(watershed_footprint), labels=opened_image)
    markers = label(local_maxi)[0]
    labels = watershed(-distance, markers, mask=opened_image)
    
    # Labeling and Sorting
    regions = regionprops_table(labels, intensity_image=img, properties=('label', 'centroid', 'max_intensity', 'mean_intensity'))
    df = pd.DataFrame(regions)
    df = df.sort_values(by=['mean_intensity','max_intensity'], ascending=False)
    
    return df

def move_to_blob(df):
    if df.shape[0] > 0:
        y=df['centroid-0'][0]*bins
        x=df['centroid-1'][0]*bins

        movex = (x-target_x)*conversion
        movey = (y-target_y)*conversion
        ce_system.xy_stage.set_rel_x(movex)
        ce_system.xy_stage.set_rel_y(-movey)
        
def limited_fire(ce_system, gravity_drop, voltage_level, injection_time):
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.high_voltage.start()
    start_time = time.time()
    ce_system.lysis_laser.laser_fire()
    ce_system.objective.set_rel_z(-0.010)

    while time.time()-start_time < injection_time:
        if time.time()-start_time < 3:
            ce_system.objective.set_rel_z(0.005)
            ce_system.xy_stage.set_rel_x(0.001)
            ce_system.lysis_laser.laser_fire()
        time.sleep(0.35)
    ce_system.high_voltage.stop()
    print(f"Finished {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)

In [27]:
# Bring the Cells into focus
fluor_channel = 3
focus_plane.get_plane_focus()


In [28]:
data_dir = r"C:\Users\Luke\Documents\Brae\9-19-2020\images"
fluor_saver = ImageSaver(ce_system, folder_prefix='Picking', data_folder=data_dir)
focus_saver = ImageSaver(ce_system, folder_prefix='Focusing', data_folder=data_dir)
lysis_saver = ImageSaver(ce_system, folder_prefix='Lysis', data_folder=data_dir)

def automatic_sampling():
    # Bring the Cells into focus
    fluor_channel = 3
    focus_plane.get_plane_focus()
    time.sleep(3)
    
    # Test Alec's Focus and get data
    pre_fluoresence()
    focus_saver.add_callback()
    focus_search = Focus.FindFocus(ce_system)
    focus_search.search_step_global()
    focus_saver.remove_callback()
    post_fluoresence()
    focus_saver.save_image()
    
    # We don't Trust Alec's function yet
    fluor_channel = 3
    focus_plane.get_plane_focus()
    
    # Get Fluoresence Image for Best chance
    fluor_channel = 1
    pre_fluoresence()
    fluor_saver.add_callback()
    img = ce_system.camera.snap()
    fluor_saver.remove_callback()
    post_fluoresence()
    
    
    # Move to the brighest blob
    blobs = get_brightest_blobs(img)
    move_to_blob(blobs)
    
    
    # Bring the Cells into focus
    fluor_channel = 3
    focus_plane.get_plane_focus()
    lysis_saver.add_callback()
    time.sleep(0.5)
    
    # on reruns, you just need to run this command when you have the cells in focus
    resp= True
    if resp:
        obj_height = ce_system.objective.read_z()
        print(obj_height)
        cap_z = get_cap_height(obj_height, cap_difference)
        z = ce_system.inlet_z.read_z()
        rel_z = cap_z-z
        rel_z += 0.02
        ce_system.inlet_z.set_rel_z(rel_z)
    else:
        print("Bring cells into focus.")
        raise RuntimeError("Cells not in Focus")

    time.sleep(5)
    
    # Start the Single Cell Injection
    limited_fire(ce_system, gravity_drop, voltage_level, injection_time)
    
    # Prepare to start the method
    ce_system.inlet_z.set_rel_z(3)
    time.sleep(3)
    lysis_saver.remove_callback()
    lysis_saver.save_image()
    
    # Start the Method
    ce_system.objective.set_rel_z(-3)
    if not auto_program.continue_event.is_set():
        auto_program.continue_event.set()
    else:
        auto_program.start_run()

NameError: name 'ImageSaver' is not defined

# Find the Brightest Cell

In [194]:
# PARAMETERS
target_x = 871
target_y = 585
rgb_chnl = 'R'
fluor_channel = 1
wait = 0.25
exp = 500
_old_exp=100
_old_bin = 1
_old_channel = 0
bins = 1
conversion =0.0003125
obj_40x = 0.0001607717041800643

def pre_fluoresence():
    # SNAP a FLUOR IMAGE
    # Set to new values
    ce_system.inlet_rgb.turn_off_channel(rgb_chnl)
    ce_system.camera.stop()
    ce_system.filter_wheel.set_channel(fluor_channel)
    ce_system.excitation_shutter.set_auto_on()
    #_old_exp = ce_system.camera.exposure
    ce_system.camera.set_exposure(exp)
    #ce_system.camera.set_binning(bins)
    #_old_bin = ce_system.camera.bin_size
    time.sleep(wait)
    
def post_fluoresence():
    # Return to old values
    ce_system.camera.set_exposure(_old_exp)
    #ce_system.camera.set_binning(_old_bin)
    ce_system.excitation_shutter.set_auto_off()
    ce_system.filter_wheel.set_channel(_old_channel)
    ce_system.inlet_rgb.turn_on_channel(rgb_chnl)

    ce_system.camera.continuous_snap()

In [271]:
# Bring the Cells into focus
fluor_channel = 3
focus_plane.get_plane_focus()


In [236]:
# Search for the best focal plane near here
pre_fluoresence()
focus_search.search_step_global()
post_fluoresence()

HEYYOO


In [258]:
fluor_channel = 1
pre_fluoresence()
img = ce_system.camera.snap()
post_fluoresence()


HEYYOO


In [184]:
plt.figure()
plt.imshow(img)

In [259]:
close_size=5
open_size=5
watershed_footprint = (5,5)
# Make sure types are the same
input_image = img_as_float(img)

# Filter Image
filtered_image = filters.median(input_image, behavior='ndimage')

# Edge Detection
edge_sobel = filters.sobel(filtered_image)

# Threshold
thresh = filters.threshold_otsu(filtered_image)
binary_otsu = filtered_image > thresh

# Binary Morphology Operations
structure_element = morphology.disk(close_size)
closed_image = morphology.binary_closing(binary_otsu, structure_element)
structure_element = morphology.disk(open_size)
opened_image = morphology.binary_opening(closed_image, structure_element)

# Watershed
distance = distance_transform_edt(opened_image)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones(watershed_footprint), labels=opened_image)
markers = label(local_maxi)[0]
labels = watershed(-distance, markers, mask=opened_image)

plt.figure()
plt.imshow(labels)

regions = regionprops_table(labels, intensity_image=img, properties=('label', 'centroid', 'max_intensity', 'mean_intensity'))


df = pd.DataFrame(regions)

df = df.sort_values(by=['mean_intensity','max_intensity'], ascending=False)

In [260]:
if df.shape[0] > 0:
    y=df['centroid-0'][0]*bins
    x=df['centroid-1'][0]*bins
    
    movex = (x-target_x)*conversion
    movey = (y-target_y)*conversion
    ce_system.xy_stage.set_rel_x(movex)
    ce_system.xy_stage.set_rel_y(-movey)

# Single Cell Injection/Load 

In [261]:
# on reruns, you just need to run this command when you have the cells in focus
resp = get_yes_no("are the cells in focus? (You may crush the capillary if not...)")
if resp:
    obj_height = ce_system.objective.read_z()
    print(obj_height)
    cap_z = get_cap_height(obj_height, cap_difference)
    z = ce_system.inlet_z.read_z()
    rel_z = cap_z-z
    rel_z += 0.02
    ce_system.inlet_z.set_rel_z(rel_z)
else:
    print("Bring cells into focus.")

are the cells in focus? (You may crush the capillary if not...) y
9.00415013417229


In [225]:
# Raise the objective for optimal plasma formation
ce_system.objective.set_rel_z(-.01)

In [139]:
# Lower the capillary (ADJUSTMENT)
ce_system.inlet_z.set_rel_z(-0.03)

In [262]:
# General Injection Parameters
injection_time = 5 # in seconds
gravity_drop = 0 # in millimeters
voltage_level = 7 # in kilovolts
delay = 1 # inseconds

In [263]:
# Start the Single Cell Injection
import threading

def limited_fire(ce_system, gravity_drop, voltage_level, injection_time):
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.high_voltage.start()
    ce_system.objective.set_rel_z(-0.02)
    start_time = time.time()
    while time.time()-start_time < injection_time:
        time.sleep(0.5)
        ce_system.objective.set_rel_z(0.005)
        ce_system.xy_stage.set_rel_x(0.001)
        ce_system.lysis_laser.laser_fire()
    ce_system.high_voltage.stop()
    print(f"Finished {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)
    

threading.Thread(target=limited_fire, args=(ce_system, gravity_drop, voltage_level, injection_time)).start()

Fire


Finished 5.284869194030762


In [244]:
resp = ""
while resp != 'q':
    resp=input("Press enter to fire, press q to exit")
    if resp != 'q':
        ce_system.lysis_laser.laser_fire()

Press enter to fire, press q to exitq


In [265]:
# Start the Method
ce_system.objective.set_rel_z(-3)
if not auto_program.continue_event.is_set():
    auto_program.continue_event.set()
else:
    auto_program.start_run()

After special True
Time to wait is  3


After special True
Time to wait is  80


# PUT YOUR CRAP FUNCTIONS HERE

In [5]:
# Start Rinse
ce_system.outlet_pressure.rinse_pressure()

In [4]:
# Stop Rinse 
ce_system.outlet_pressure.release()

In [4]:
ce_system.lysis_laser.startup()

In [78]:
laser = ce_system.lysis_laser

In [292]:
laser.laser_standby()

In [291]:
laser.laser_stop()

In [293]:
laser.laser_fire()

In [11]:
ce_system.objective.set_rel_z(-3)

In [127]:
ce_system.inlet_rgb.turn_on_channel('R')


'R:On, G:Off, B:Off'

In [126]:
ce_system.inlet_rgb.turn_off_channel('R')


'R:Off, G:Off, B:Off'

In [58]:
xy = ce_system.xy_stage.read_xy()
xy

[28.52200042501092, -1.8983000282868743]

In [68]:
ce_system.xy_stage.set_xy(xy)

In [10]:
ce_system.inlet_z.set_z(10)

In [147]:
ce_system.outlet_pressure.seal()

In [6]:
cam = ce_system.camera

In [7]:
ce_system.inlet_z.controller.open()

In [9]:
ce_system.inlet_z.startup()

In [3]:
output = r"C:\Users\Luke\Documents\Data\images"
im_sav = ImageSaver(ce_system, data_folder=output, prefix='testing')

In [4]:
im_sav.add_callback()

In [7]:
cam.continuous_snap()

HEYYOO


True

In [8]:
cam.stop()

True

In [9]:
im_sav.remove_callback()

In [14]:
im_sav.save_image(unique_folder=False)

C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img_00000.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img_00001.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img_00002.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img_00003.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\i